In [ ]:
!pip install datasets sentence-transformers faiss-cpu langchain openai
!pip install sacremoses
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 22.4 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("KryptoniteCrown/synthetic-neurology-QA-dataset", split = "train")

neurology_qa = [{"question": row["question"], "answer": row["answer"]} for row in dataset]

print(neurology_qa[0])

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

reformatted_data_neurology.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1452 [00:00<?, ? examples/s]

{'question': 'What are the key characteristics of deep dyslexia?', 'answer': 'Deep dyslexia is characterized by both semantic paralexia (paralexia) and phonological dyslexia. Individuals with this condition may read a word and substitute it with a semantically related word. It involves significant phonological processing impairments and is often caused by extensive brain damage, frequently in the left hemisphere.'}


In [ ]:
cardiology_dataset = load_dataset("ilyassacha/cardiology_qa", split="train")
cardiology_qa = [{"question": row["question"], "answer": row["answer"]} for row in cardiology_dataset]
print("Cardiology Q&A samples:", len(cardiology_qa))
print(cardiology_qa[0])

README.md:   0%|          | 0.00/438 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/11.7M [00:00<?, ?B/s]

data/eval-00000-of-00001.parquet:   0%|          | 0.00/1.35M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14885 [00:00<?, ? examples/s]

Generating eval split:   0%|          | 0/1654 [00:00<?, ? examples/s]

Cardiology Q&A samples: 14885
{'question': 'My husband of 82 years is on multiple medications for heart disease and Parkensons. His feet are very swollen ...is this because he is retaining fluids and if so what can he do to relieve this. He is under the care of several doctors but they have not offered or explained or shed any light on the cause. margaretmidea@ icloud.com', 'answer': 'hi, thank-you for the brief history.to reduce the retention there should be some muscular activity. it will be active or passive. a physical therapist will play a key role. also keeping the legs elevated will be good for reducing some swelling. use lower limb stockings for compression effect. as due to lack of mobility in the muscular system this is happening. i wish you find a physical therapist who can take the responsibility further and help your husband to the best.  with the grace of god i wish you a speedy recoveryregardsjay in chatbot.'}


In [ ]:
dermatology_dataset = load_dataset("Mreeb/Dermatology-Question-Answer-Dataset-For-Fine-Tuning", split="train")
dermatology_qa = [{"question": row["prompt"], "answer": row["response"]} for row in dermatology_dataset]
print("Dermatology Q&A samples:", len(dermatology_qa))
print(dermatology_qa[0])

README.md:   0%|          | 0.00/929 [00:00<?, ?B/s]

combined_data.csv: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/1460 [00:00<?, ? examples/s]

Dermatology Q&A samples: 1460
{'question': 'What is psoriasis and what are its common symptoms?', 'answer': 'Psoriasis is a chronic autoimmune condition that results in the overproduction of skin cells. This overproduction leads to patches of thick, red skin covered with silvery scales. Common symptoms include red patches of skin covered with thick, silvery scales, small scaling spots (commonly seen in children), dry and cracked skin that may bleed, itching, burning, or soreness, thickened, pitted, or ridged nails, and swollen and stiff joints.'}


In [ ]:
breast_cancer_dataset = load_dataset("grasool/breast-cancer-QAs-llama", split = "train")
breast_cancer_qa = []
for row in breast_cancer_dataset:
  text = row["text"]

  text = text.replace("<s>", "").replace("</s>", "").strip()

  if "[/INST]" in text :
    parts = text.split("[/INST]")
    question_part = parts[0].replace("[INST]", "").strip()
    answer_part = parts[1].strip()

    breast_cancer_qa.append({"question": question_part, "answer": answer_part})

print("Breast Cancer Q&A samples:", len(breast_cancer_qa))
print(breast_cancer_qa[0])

README.md:   0%|          | 0.00/541 [00:00<?, ?B/s]

data/train-00000-of-00001-c0696c2b7ee38e(…):   0%|          | 0.00/56.6k [00:00<?, ?B/s]

data/test-00000-of-00001-1fb925721582029(…):   0%|          | 0.00/9.27k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/298 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/34 [00:00<?, ? examples/s]

Breast Cancer Q&A samples: 298
{'question': 'How can I create a healthy diet plan as a cancer survivor?', 'answer': 'As a cancer survivor, focus on eating whole, unprocessed foods. Consult a nutritionist if possible and develop a meal plan tailored to your needs and preferences. A healthy diet can aid recovery, improve metabolism, and support your overall well-being.'}


In [ ]:
from google.colab import files
import pandas as pd

uploaded = files.upload()
filename = list(uploaded.keys())[0]

df = pd.read_csv(filename)
display(df)

cancer_dataset = [{"question": row["Question"], "answer": row["Answer"]} for _, row in df.iterrows()]
print("Cancer Q&A samples:", len(cancer_dataset))
print(cancer_dataset[0])

Saving CancerQA.csv to CancerQA (3).csv


,Question,Answer,topic,split
0,What is (are) Non-Small Cell Lung Cancer ?,Non-small cell lung cancer is a disease in whi...,cancer,train
1,Who is at risk for Non-Small Cell Lung Cancer? ?,Smoking is the major risk factor for non-small...,cancer,train
2,What are the symptoms of Non-Small Cell Lung C...,Signs of non-small cell lung cancer include a ...,cancer,test
3,How to diagnose Non-Small Cell Lung Cancer ?,Tests that examine the lungs are used to detec...,cancer,train
4,What is the outlook for Non-Small Cell Lung Ca...,Certain factors affect prognosis (chance of re...,cancer,train
...,...,...,...,...
724,How to diagnose Adult Soft Tissue Sarcoma ?,Adult soft tissue sarcoma is diagnosed with a ...,cancer,test
725,What is the outlook for Adult Soft Tissue Sarc...,Certain factors affect treatment options and p...,cancer,test
726,What are the stages of Adult Soft Tissue Sarco...,Key Points\n - After adult ...,cancer,train
727,what research (or clinical trials) is being do...,Patients may want to think about taking part i...,cancer,train


Cancer Q&A samples: 729
{'question': 'What is (are) Non-Small Cell Lung Cancer ?', 'answer': "Non-small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.     - There are several types of non-small cell lung cancer.    - Smoking is the major risk factor for non-small cell lung cancer.    - Signs of non-small cell lung cancer include a cough that doesn't go away and shortness of breath.    - Tests that examine the lungs are used to detect (find), diagnose, and stage non-small cell lung cancer.    - Certain factors affect prognosis (chance of recovery) and treatment options.    - For most patients with non-small cell lung cancer, current treatments do not cure the cancer.\n                \n                \n                    Non-small cell lung cancer is a disease in which malignant (cancer) cells form in the tissues of the lung.\n                    The lungs are a pair of cone-shaped breathing organs in the chest. The lungs bring oxygen 

In [ ]:
embedder = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def build_faiss_index(docs, domain_name=""):
    texts = [d["question"] for d in docs]
    embeddings = embedder.encode(texts, convert_to_numpy=True, show_progress_bar=True)
    dim = embeddings.shape[1]
    index = faiss.IndexFlatL2(dim)
    index.add(embeddings)

    # Print vector count for this domain
    print(f"[{domain_name}] Total vectors stored: {index.ntotal}")

    return index, docs

# Build indexes for each dataset
neurology_index, neurology_docs = build_faiss_index(neurology_qa, "Neurology")
cardio_index, cardio_docs = build_faiss_index(cardiology_qa, "Cardiology")
derma_index, derma_docs = build_faiss_index(dermatology_qa, "Dermatology")
breast_cancer_index, breast_cancer_docs = build_faiss_index(breast_cancer_qa, "Breast Cancer")
cancer_index, cancer_docs = build_faiss_index(cancer_dataset, "Cancer")

# Store in dictionary
vector_dbs = {
    "Neurology": (neurology_index, neurology_docs),
    "Cardiology": (cardio_index, cardio_docs),
    "Dermatology": (derma_index, derma_docs),
    "Breast Cancer": (breast_cancer_index, breast_cancer_docs),
    "Cancer": (cancer_index, cancer_docs),
}

# Print a final summary
print("\nSummary of Vector DBs:")
for domain, (index, docs) in vector_dbs.items():
    print(f" - {domain.capitalize()}: {index.ntotal} vectors")

Batches:   0%|          | 0/46 [00:00<?, ?it/s]

[Neurology] Total vectors stored: 1452


Batches:   0%|          | 0/466 [00:00<?, ?it/s]

[Cardiology] Total vectors stored: 14885


Batches:   0%|          | 0/46 [00:00<?, ?it/s]

[Dermatology] Total vectors stored: 1460


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

[Breast Cancer] Total vectors stored: 298


Batches:   0%|          | 0/23 [00:00<?, ?it/s]

[Cancer] Total vectors stored: 729

Summary of Vector DBs:
 - Neurology: 1452 vectors
 - Cardiology: 14885 vectors
 - Dermatology: 1460 vectors
 - Breast cancer: 298 vectors
 - Cancer: 729 vectors


In [ ]:
def route_pipeline(query: str):
    q = query.lower().strip()  # normalize the query text

    cardiology_keywords = [
        # 1. Cardiovascular Diseases and Conditions
        "heart disease", "cardiovascular disease", "coronary artery disease", "cad",
        "myocardial infarction", "heart attack", "angina", "unstable angina",
        "stable angina", "heart failure", "congestive heart failure", "chf",
        "arrhythmia", "atrial fibrillation", "afib", "ventricular tachycardia",
        "vt", "bradycardia", "tachycardia", "cardiomyopathy", "hypertrophic cardiomyopathy",
        "dilated cardiomyopathy", "restrictive cardiomyopathy", "pericarditis",
        "endocarditis", "myocarditis", "valvular disease", "aortic stenosis",
        "mitral regurgitation", "aortic regurgitation", "mitral stenosis",
        "pulmonary hypertension", "systemic hypertension", "high blood pressure",
        "low blood pressure", "hypotension", "cor pulmonale", "cardiac arrest",
        "sudden cardiac death", "ischemic heart disease", "atherosclerosis",
        "hyperlipidemia", "dyslipidemia", "high cholesterol",
        # 2. Heart Anatomy and Physiology
        "heart", "aorta", "atrium", "ventricle", "mitral valve", "aortic valve",
        "tricuspid valve", "pulmonary valve", "coronary arteries", "left ventricle",
        "right ventricle", "left atrium", "right atrium", "septum", "endocardium",
        "myocardium", "pericardium", "sinus node", "sa node", "av node", "bundle of his",
        "purkinje fibers", "blood vessel", "artery", "vein", "capillary",
        # 3. Diagnostic Tests and Imaging
        "ecg", "ekg", "electrocardiogram", "echocardiogram", "echo", "stress test",
        "treadmill test", "angiogram", "cardiac catheterization", "coronary angiography",
        "ct angiography", "mri heart", "holter monitor", "event monitor",
        "cardiac mri", "cardiac ct", "nuclear stress test", "tilt table test",
        "blood pressure monitor", "troponin", "cardiac enzymes",
        # 4. Treatments, Procedures, and Medications
        "angioplasty", "stent", "coronary stent", "bypass surgery", "cabg",
        "pacemaker", "implantable cardioverter defibrillator", "icd",
        "ablation", "valve replacement", "valve repair", "heart transplant",
        "cardioversion", "defibrillation", "thrombolysis", "anticoagulant",
        "warfarin", "heparin", "aspirin", "beta blocker", "ace inhibitor",
        "arb", "statin", "calcium channel blocker", "diuretic", "nitrate",
        "vasodilator", "antiplatelet", "antihypertensive", "digoxin",
        "clopidogrel", "tpa", "nitroglycerin",
        # 5. Symptoms and Clinical Signs
        "chest pain", "palpitations", "shortness of breath", "dyspnea",
        "fatigue", "syncope", "fainting", "edema", "swelling", "cyanosis",
        "orthopnea", "paroxysmal nocturnal dyspnea", "leg swelling",
        "dizziness", "lightheadedness", "heart murmur",
        # 6. Risk Factors and Related Terms
        "smoking", "obesity", "diabetes", "high cholesterol", "hypercholesterolemia",
        "hypertension", "stress", "family history", "age", "male gender",
        "sedentary lifestyle", "poor diet", "alcohol", "metabolic syndrome",
        # 7. Subspecialties and Related Fields
        "interventional cardiology", "electrophysiology", "cardiac surgery",
        "preventive cardiology", "nuclear cardiology", "pediatric cardiology",
        "vascular medicine", "cardiac rehabilitation", "heart rhythm disorders",
        # 8. Acronyms and Common Abbreviations
        "cad", "chf", "afib", "vt", "ecg", "ekg", "cabg", "icd", "sa node",
        "av node", "lv", "rv", "bp", "hdl", "ldl", "tpa", "murmur"
    ]

    dermatology_keywords = [
        # 1. Common Skin Diseases and Conditions
    "acne", "rosacea", "eczema", "dermatitis", "atopic dermatitis", "contact dermatitis",
    "seborrheic dermatitis", "psoriasis", "urticaria", "hives", "alopecia", "hair loss",
    "vitiligo", "melasma", "hyperpigmentation", "hypopigmentation", "lupus",
    "scleroderma", "fungal infection", "tinea", "ringworm", "athlete's foot", "onychomycosis",
    "nail fungus", "warts", "verruca", "molluscum contagiosum", "impetigo", "cellulitis",
    "abscess", "boil", "carbuncle", "hidradenitis suppurativa", "scabies", "lice",
    "shingles", "herpes zoster", "cold sore", "herpes simplex", "chickenpox",
    "basal cell carcinoma", "squamous cell carcinoma", "melanoma", "skin cancer",
    "actinic keratosis", "sunburn", "contact allergy", "drug eruption", "blister",
    "rash", "itching", "pruritus", "hives", "dry skin", "seborrhea", "corn", "callus",
    "eczema herpeticum", "lichen planus", "pityriasis rosea", "dermatophytosis",

    # 2. Skin Anatomy and Structures
    "skin", "epidermis", "dermis", "subcutaneous tissue", "sweat gland", "sebaceous gland",
    "hair follicle", "nail", "melanocyte", "keratinocyte", "collagen", "elastin",
    "pores", "sebaceous glands", "oil gland",

    # 3. Diagnostic Tests and Procedures
    "skin biopsy", "dermoscopy", "woods lamp", "skin scraping", "patch test",
    "skin culture", "allergy test", "trichoscopy", "nail clipping test",
    "histopathology", "microscopy",

    # 4. Treatments, Procedures, and Medications
    "topical steroid", "corticosteroid", "antifungal cream", "antibiotic ointment",
    "moisturizer", "emollient", "retinoid", "benzoyl peroxide", "isotretinoin",
    "accutane", "phototherapy", "laser therapy", "cryotherapy", "electrocautery",
    "skin graft", "chemical peel", "microdermabrasion", "botox", "filler",
    "antihistamine", "antiviral cream", "steroid injection", "coal tar",
    "calcineurin inhibitor", "tacrolimus", "pimecrolimus", "zinc oxide",
    "salicylic acid", "sunscreen", "spf", "uv protection", "hydration cream",

    # 5. Symptoms and Clinical Signs
    "rash", "redness", "itching", "pruritus", "swelling", "blister", "pustule",
    "papule", "nodule", "plaque", "scaling", "flaking", "dryness", "peeling",
    "lesion", "ulcer", "erosion", "crust", "scab", "pigmentation", "spot",
    "mole", "nevus", "lump", "bump", "discoloration", "burn", "scar", "keloid",

    # 6. Cosmetic and Aesthetic Dermatology
    "acne scars", "hyperpigmentation", "chemical peel", "laser resurfacing",
    "anti-aging", "wrinkle", "botox", "filler", "skin rejuvenation",
    "microneedling", "dermal filler", "facial treatment", "skin whitening",
    "pigmentation removal", "tattoo removal",

    # 7. Subspecialties and Related Areas
    "cosmetic dermatology", "pediatric dermatology", "surgical dermatology",
    "immunodermatology", "trichology", "dermatopathology", "venereology",

    # 8. Acronyms and Common Abbreviations
    "spf", "bsa", "uv", "uvb", "uva", "nmsc", "bcc", "scc", "bpo"
    ]

    neurology_keywords = [
        # 1. Neurological Conditions
    "stroke", "migraine", "epilepsy", "seizure", "multiple sclerosis", "ms",
    "parkinson", "parkinson's", "alzheimer", "dementia", "huntington", "als",
    "lou gehrig", "neuropathy", "neuralgia", "radiculopathy", "meningitis",
    "encephalitis", "brain tumor", "glioma", "astrocytoma", "concussion",
    "traumatic brain injury", "tbi", "spinal cord injury", "dystonia", "tremor",
    "headache", "cluster headache", "vertigo", "ataxia", "neuropathic pain",
    "sciatica",

    # 2. Brain and Nervous System Anatomy
    "brain", "spinal cord", "neuron", "nerve", "synapse", "axon", "dendrite",
    "cortex", "cerebellum", "brainstem", "medulla", "thalamus", "hypothalamus",
    "basal ganglia", "frontal lobe", "temporal lobe", "parietal lobe",
    "occipital lobe", "cerebrum", "meninges", "peripheral nervous system",
    "central nervous system", "cns", "pns",

    # 3. Diagnostic Tests and Imaging
    "mri", "ct scan", "eeg", "emg", "nerve conduction study", "lumbar puncture",
    "spinal tap", "brain scan", "neuroimaging", "pet scan", "spect",
    "angiography", "evoked potentials",

    # 4. Treatments, Procedures, and Medications
    "deep brain stimulation", "dbs", "neurorehabilitation", "physiotherapy",
    "antiepileptic", "levodopa", "dopamine agonist", "anticonvulsant",
    "neurologist", "neurosurgery", "neurofeedback", "botulinum toxin",
    "tpa", "thrombolysis",

    # 5. Symptoms and Clinical Signs
    "numbness", "tingling", "weakness", "paralysis", "memory loss", "confusion",
    "tremor", "balance problems", "speech difficulty", "dizziness", "fainting",
    "cognitive decline", "vision changes", "muscle spasm", "coordination loss",

    # 6. Related Subfields
    "neuroscience", "neurobiology", "neuropsychology", "cognitive neuroscience",
    "behavioral neurology", "neuroanatomy", "neurophysiology", "neuropharmacology",

    # 7. Acronyms and Common Abbreviations
    "cns", "pns", "ms", "als", "tbi", "ad", "pd", "eeg", "emg", "dbs",
    "mri", "ct", "lp", "csf", "cva"
    ]

    breast_cancer_keywords = [
        "breast cancer", "carcinoma", "tumor", "lump", "mass", "malignancy", "metastasis", "oncology",
        "mammary", "ductal", "lobular", "her2", "triple-negative", "er-positive", "pr-positive",
        "nipple discharge", "nipple inversion", "breast pain", "skin dimpling", "swelling",
        "breast tenderness", "armpit lump", "axillary swelling",
        "mammogram", "biopsy", "ultrasound", "mri", "pet scan", "screening", "brca", "genetic test",
        "pathology", "histopathology", "radiologist",
        "chemotherapy", "radiation", "mastectomy", "lumpectomy", "reconstruction",
        "immunotherapy", "hormone therapy", "targeted therapy", "tamoxifen", "anastrozole",
        "letrozole", "trastuzumab", "herceptin", "paclitaxel", "cisplatin", "carboplatin",
        "stage", "grade", "invasive", "ductal carcinoma", "lobular carcinoma", "remission",
        "recurrence", "prognosis", "survival rate",
        "self-exam", "screening", "diet", "nutrition", "exercise", "obesity", "alcohol", "smoking",
        "menopause", "hormone replacement therapy", "birth control",
        "brca1", "brca2", "gene mutation", "inherited", "familial", "clinical trial",
        "survivor", "support group", "mental health", "counseling", "rehabilitation", "fatigue"
    ]

    cancer_keywords = [
    # General terms
    "cancer", "tumor", "malignant", "benign", "oncology", "oncologist",
    "neoplasm", "carcinoma", "sarcoma", "leukemia", "lymphoma", "melanoma",
    "metastasis", "metastatic", "lesion", "mass", "growth", "nodule",

    # Types of cancer
    "breast cancer", "lung cancer", "colon cancer", "colorectal cancer",
    "skin cancer", "melanoma", "pancreatic cancer", "prostate cancer",
    "ovarian cancer", "cervical cancer", "uterine cancer", "brain tumor",
    "glioblastoma", "thyroid cancer", "kidney cancer", "renal carcinoma",
    "bladder cancer", "stomach cancer", "gastric cancer", "liver cancer",
    "hepatocellular carcinoma", "esophageal cancer", "oral cancer",
    "blood cancer", "multiple myeloma", "endometrial cancer",

    # Symptoms
    "lump", "mass", "nodule", "pain", "fatigue", "weight loss", "swelling",
    "bleeding", "nausea", "vomiting", "anemia", "appetite loss",
    "breathing difficulty", "persistent cough", "blood in stool",
    "night sweats", "bone pain", "jaundice", "skin discoloration",

    # Diagnosis & screening
    "biopsy", "ct scan", "mri", "pet scan", "x-ray", "mammogram",
    "ultrasound", "screening", "diagnosis", "early detection", "pathology",
    "histology", "stage 1", "stage 2", "stage 3", "stage 4", "grading",
    "tumor markers", "brca", "genetic testing", "mutation", "oncogene",
    "tumor suppressor", "carcinoembryonic antigen", "screening test",

    # Treatments
    "chemotherapy", "radiation therapy", "radiotherapy", "immunotherapy",
    "targeted therapy", "hormone therapy", "surgery", "mastectomy",
    "lumpectomy", "resection", "palliative care", "clinical trial",
    "drug therapy", "oncology drugs", "car-t", "checkpoint inhibitors",
    "nivolumab", "pembrolizumab", "imatinib", "trastuzumab", "hormonal therapy",

    # Recovery & survivorship
    "cancer survivor", "remission", "recurrence", "follow-up", "rehabilitation",
    "quality of life", "cancer support", "nutrition", "diet plan", "cancer care",

    # Risk & causes
    "risk factors", "smoking", "alcohol", "radiation exposure", "hpv", "hepatitis",
    "genetics", "family history", "obesity", "pollution", "occupational hazard",

    # Related medical contexts
    "oncologic", "cytology", "metastatic spread", "tumor cells", "cell division",
    "angiogenesis", "tumor microenvironment", "immune response", "apoptosis",

    # Prevention & lifestyle
    "screening program", "self-exam", "mammogram", "pap smear", "colonoscopy",
    "early signs", "healthy lifestyle", "dietary changes", "exercise", "smoking cessation",
]


    # Routing logic
    if any(word in q for word in cardiology_keywords):
        print("Cardiology")
        return ["Cardiology"]
    elif any(word in q for word in dermatology_keywords):
        print("Dermatology")
        return ["Dermatology"]
    elif any(word in q for word in neurology_keywords):
        print("Neurology")
        return ["Neurology"]
    elif any(word in q for word in breast_cancer_keywords):
        print("Breast Cancer")
        return ["Breast_cancer"]
    elif any(word in q for word in cancer_keywords):
        print("Cancer")
        return ["Cancer"]
    else:
        print("Please ask a question related to the medical domain.")
        return []

In [ ]:
import torch
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSeq2SeqLM
import time
import random

In [ ]:
hyde_model_name = "microsoft/BioGPT"
hyde_tokenizer = AutoTokenizer.from_pretrained(hyde_model_name)
hyde_model = AutoModelForCausalLM.from_pretrained(hyde_model_name)

reranker_model_name = "google/flan-t5-large"
reranker_tokenizer = AutoTokenizer.from_pretrained(reranker_model_name)
reranker_model = AutoModelForSeq2SeqLM.from_pretrained(reranker_model_name)

config.json:   0%|          | 0.00/595 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.56G [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
def hyde_hypothesis(query, max_new_tokens=120):
    """
    Generate a clean hypothetical medical answer (HyDE step).
    """
    # prompt = f"""
    # You are a senior medical expert. Given the following question, create a hypothetical but medically valid answer.
    # Make it structured in this exact format:

    # Causes:
    # - <cause 1>
    # - <cause 2>
    # - <cause 3>

    # Treatments:
    # - <treatment 1>
    # - <treatment 2>
    # - <treatment 3>

    # Follow-up:
    # - <test or next step 1>
    # - <test or next step 2>

    # Summary:
    # <2–3 sentence summary>

    # Question: {query}
    # """

    prompt = f"""
You are a medical expert. Given the following question, generate a concise, evidence-based hypothetical answer.
Avoid creative writing. Stick to factual, clinical reasoning only.
Question: {query}
Answer:
"""
    print("\n Generating hypothetical medical answer...")
    inputs = hyde_tokenizer(prompt, return_tensors="pt", truncation=True)
    outputs = hyde_model.generate(**inputs, max_new_tokens=max_new_tokens, temperature=0.4, do_sample=False)
    hypo_answer = hyde_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
    hypo_answer = hypo_answer.replace("< / FREETEXT >", "").replace("< / ABSTRACT >", "").strip()
    print(hypo_answer)
    return hypo_answer.strip()


In [ ]:
def llm_rerank(query, candidate_answers):
    """
    Evaluate and rank candidate answers using an LLM-based reranker.
    """
    scored_answers = []

    for ans in candidate_answers:
        prompt = f"""
You are a senior medical evaluator.
Evaluate how well the candidate answer addresses the medical question below.

Question: {query}

Candidate Answer: {ans}

Give a score:
- 1.0 if the answer directly and completely addresses the medical question accurately.
- 0.5 if it's partially related but missing key information or clarity.
- 0.0 if it's irrelevant, incorrect, or unrelated.

Output only the number (0.0, 0.5, or 1.0) — no explanation.
"""
        print("\n LLM RERANKER Evaluating candidate answer...")
        inputs = reranker_tokenizer(prompt, return_tensors="pt", truncation=True)
        outputs = reranker_model.generate(**inputs, max_new_tokens=5)
        score_text = reranker_tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

        try:
            score = float(score_text)
        except ValueError:
            score = 0.0
            print(f" Could not parse score '{score_text}', defaulting to 0.0")

        scored_answers.append({"answer": ans, "reranker_score": score})

    # Sort by reranker score
    ranked = sorted(scored_answers, key=lambda x: x["reranker_score"], reverse=True)
    print("\n [RERANKER RESULTS]")
    for i, r in enumerate(ranked):
        print(f"{i+1}. Score: {r['reranker_score']} | Answer snippet: {r['answer'][:120]}...")

    return ranked



In [ ]:
def retrieve_answer(query, k=3, use_hyde=True):
    """Complete retrieval and reranking pipeline, returning ONLY the best answer."""
    if use_hyde:
        query_text = hyde_hypothesis(query)
    else:
        query_text = query

    selected_dbs = route_pipeline(query)
    query_emb = embedder.encode([query_text])
    candidates = []

    for db_name in selected_dbs:
        index, docs = vector_dbs[db_name]
        distances, indices = index.search(np.array(query_emb), k)
        candidates.extend([docs[i]["answer"] for i in indices[0]])

    results = llm_rerank(query, candidates)

    best_answer = results[0]["answer"]

    print("\n[BEST MEDICAL ANSWER]\n")
    print(best_answer)

    return {
        "hyde_hypothesis": query_text,
        "best_retrieved_answer": best_answer,
        "ranked_results": results
    }


In [ ]:
query = "What are the side effects of chemotherapy?"
results = retrieve_answer(query, k=3, use_hyde=True)

show_debug = False  # change to True to see all results
if show_debug:
    print("\nRanked Results:")
    for r in results["ranked_results"]:
        print(f"Score: {r['reranker_score']:.2f} | Answer snippet: {r['answer'][0 : ]}...")

The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.



 Generating hypothetical medical answer...
You are a medical expert. Given the following question, generate a concise, evidence-based hypothetical answer. Avoid creative writing. Stick to factual, clinical reasoning only. Question: What are the side effects of chemotherapy? Answer: Chemotherapy can cause a variety of side effects, including nausea, vomiting, alopecia, and myelosuppression.
Neurology

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 [RERANKER RESULTS]
1. Score: 1.0 | Answer snippet: Genetic mutations can have harmful, beneficial, neutral (no effect), or uncertain effects on a person's health. Harmful ...
2. Score: 1.0 | Answer snippet: Patients with anosognosia may not perceive the need for medication, leading to non-compliance. Furthermore, the side eff...
3. Score: 0.0 | Answer snippet: Awareness of illness in psychiatric patients with anosognosia may improve with treatment usin

In [ ]:
query = "What are the early signs of breast cancer?"
results = retrieve_answer(query, k=3, use_hyde=True)

show_debug = False  # change to True to see all results
if show_debug:
    print("\nRanked Results:")
    for r in results["ranked_results"]:
        print(f"Score: {r['reranker_score']:.2f} | Answer snippet: {r['answer'][0 : ]}...")


 Generating hypothetical medical answer...

 HyDe Output
You are a senior medical expert. Given the following question, create a hypothetical but medically valid answer. Make it structured in this exact format: Causes: - < cause 1 > - < cause 2 > - < cause 3 > Treatments: - < treatment 1 > - < treatment 2 > - < treatment 3 > Follow-up: - < test or next step 1 > - < test or next step 2 > Summary: < 2 3 sentence summary > Question: What are the early signs of breast cancer?

Breast Cancer

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 [RERANKER RESULTS]
1. Score: 0.5 | Answer snippet: To gain a comprehensive understanding of your diagnosis and treatment plan, consider asking about the type of breast can...
2. Score: 0.5 | Answer snippet: For young children, use simple language and concepts they can understand. You can describe it as something growing insid...
3. Score: 0.5 | Answer snippet: Brea

In [ ]:
query = "Is chemotherapy necessary for triple-negative cancer?"
results = retrieve_answer(query, k=3, use_hyde=True)

show_debug = False  # change to True to see all results
if show_debug:
    print("\nRanked Results:")
    for r in results["ranked_results"]:
        print(f"Score: {r['reranker_score']:.2f} | Answer snippet: {r['answer'][0 : ]}...")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.



 Generating hypothetical medical answer...

 HyDe Output
You are a senior medical expert. Given the following question, create a hypothetical but medically valid answer. Make it structured in this exact format: Causes: - < cause 1 > - < cause 2 > - < cause 3 > Treatments: - < treatment 1 > - < treatment 2 > - < treatment 3 > Follow-up: - < test or next step 1 > - < test or next step 2 > Summary: < 2 3 sentence summary > Question: Is chemotherapy necessary for triple-negative cancer?

Breast Cancer

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 [RERANKER RESULTS]
1. Score: 0.5 | Answer snippet: Long-term effects might include impacts on hormones and potential weight gain. Regular checkups with your oncology cardi...
2. Score: 0.5 | Answer snippet: Patients considering chemotherapy often have questions. They wonder about its necessity, potential surgery combined with...
3. Score: 0.5 | Answer sn

In [ ]:
query = "What is the best treatment for psoriasis?"
results = retrieve_answer(query, k=3, use_hyde=True)

show_debug = False  # change to True to see all results
if show_debug:
    print("\nRanked Results:")
    for r in results["ranked_results"]:
        print(f"Score: {r['reranker_score']:.2f} | Answer snippet: {r['answer'][0 : ]}...")




 Generating hypothetical medical answer...
You are a medical expert. Given the following question, generate a concise, evidence-based hypothetical answer. Avoid creative writing. Stick to factual, clinical reasoning only. Question: What is the best treatment for psoriasis? Answer: Topical corticosteroids are the best treatment for psoriasis.
Dermatology

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 [RERANKER RESULTS]
1. Score: 1.0 | Answer snippet: Psoriasis treatment aims to stop skin cells from growing so quickly and to remove scales. Options include:

1. Topical t...
2. Score: 1.0 | Answer snippet: Treatment options for psoriasis depend on the severity and extent of the skin involvement. Topical treatments, such as c...
3. Score: 1.0 | Answer snippet: Psoriasis treatment typically aims to stop skin cells from growing so quickly and to remove scales. Topical treatments i...

[BEST MEDICAL A

In [ ]:
query = "I have been experiencing chest pain and irregular heartbeat, could this be related to my heart condition?"
results = retrieve_answer(query, k=3, use_hyde=True)

show_debug = False  # change to True to see all results
if show_debug:
    print("\nRanked Results:")
    for r in results["ranked_results"]:
        print(f"Score: {r['reranker_score']:.2f} | Answer snippet: {r['answer'][0 : ]}...")



 Generating hypothetical medical answer...

 HyDe Output
You are a senior medical expert. Given the following question, create a hypothetical but medically valid answer. Make it structured in this exact format: Causes: - < cause 1 > - < cause 2 > - < cause 3 > Treatments: - < treatment 1 > - < treatment 2 > - < treatment 3 > Follow-up: - < test or next step 1 > - < test or next step 2 > Summary: < 2 3 sentence summary > Question: I have been experiencing chest pain and irregular heartbeat, could this be related to my heart condition?

cardiology

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 [RERANKER RESULTS]
1. Score: 1.0 | Answer snippet: thanks for your question on chatbot. i can understand your concern. yes, these symptoms can be due to cardiac (heart rel...
2. Score: 1.0 | Answer snippet: hello and thank you for using chatbot. i carefully read your question and i totally understand your 

In [ ]:
query = "How does left ventricular hypertrophy influence the prognosis of hypertensive patients?"
results = retrieve_answer(query, k=3, use_hyde=True)

show_debug = False  # change to True to see all results
if show_debug:
    print("\nRanked Results:")
    for r in results["ranked_results"]:
        print(f"Score: {r['reranker_score']:.2f} | Answer snippet: {r['answer'][0 : ]}...")


 Generating hypothetical medical answer...

 HyDe Output
You are a senior medical expert. Given the following question, create a hypothetical but medically valid answer. Make it structured in this exact format: Causes: - < cause 1 > - < cause 2 > - < cause 3 > Treatments: - < treatment 1 > - < treatment 2 > - < treatment 3 > Follow-up: - < test or next step 1 > - < test or next step 2 > Summary: < 2 3 sentence summary > Question: How does left ventricular hypertrophy influence the prognosis of hypertensive patients?

Neurology

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 [RERANKER RESULTS]
1. Score: 0.5 | Answer snippet: Risk factors include conditions that affect blood vessels, such as tobacco smoking, high blood pressure, atrial fibrilla...
2. Score: 0.0 | Answer snippet: Other medical conditions, such as vitamin deficiencies, infections, or thyroid disorders, can mimic the symptoms of dem

In [ ]:
query = "I developed a circular rash on my arm that keeps growing. Could this be a fungal infection?"
results = retrieve_answer(query, k=3, use_hyde=True)
show_debug = False  # change to True to see all results
if show_debug:
    print("\nRanked Results:")
    for r in results["ranked_results"]:
        print(f"Score: {r['reranker_score']:.2f} | Answer snippet: {r['answer'][0 : ]}...")


 Generating hypothetical medical answer...

 HyDe Output
You are a senior medical expert. Given the following question, create a hypothetical but medically valid answer. Make it structured in this exact format: Causes: - < cause 1 > - < cause 2 > - < cause 3 > Treatments: - < treatment 1 > - < treatment 2 > - < treatment 3 > Follow-up: - < test or next step 1 > - < test or next step 2 > Summary: < 2 3 sentence summary > Question: I developed a circular rash on my arm that keeps growing. Could this be a fungal infection?

dermatology

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 LLM RERANKER Evaluating candidate answer...

 [RERANKER RESULTS]
1. Score: 1.0 | Answer snippet: Fungal skin infections are caused by different types of fungi, including dermatophytes and yeasts. They usually develop ...
2. Score: 1.0 | Answer snippet: Yes, a fungal nail infection can spread to other parts of the body. This is particularly common in individuals wi